In [ ]:
import numpy as np
import pandas as pd
import joblib
import json

#Savepath
save_path = r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\y_randomization_results_3T3.xlsx"

# Load the model
model_base = joblib.load(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\model_LGBM_216f.joblib")

# Load the external data
with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\curated_data\x\binary\x_ext\x_ext_ecfp_fp_2_1024.json", 'r') as f:
    x_test = json.load(f)

with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\curated_data\y\binary\y_ext\y_ext_ecfp_fp_2_1024.json", 'r') as f:
    y_test = json.load(f)

# Load the internal data
with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\curated_data\x\binary\x_train\x_train_ecfp_fp_2_1024.json", 'r') as f:
    x_train = json.load(f)

with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\3T3\curated_data\y\binary\y_train\y_train_ecfp_fp_2_1024.json", 'r') as f:
    y_train = json.load(f)



In [ ]:
X_ext_infos_df = pd.DataFrame(x_test)
X_ext_infos_df = X_ext_infos_df.add_prefix('var_')

X_int_infos_df = pd.DataFrame(x_train)
X_int_infos_df = X_int_infos_df.add_prefix('var_')


features_sel = np.array(['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_14', 'var_15',
       'var_23', 'var_24', 'var_29', 'var_33', 'var_36', 'var_41',
       'var_42', 'var_43', 'var_45', 'var_46', 'var_64', 'var_65',
       'var_74', 'var_75', 'var_77', 'var_80', 'var_83', 'var_85',
       'var_90', 'var_92', 'var_95', 'var_114', 'var_117', 'var_119',
       'var_121', 'var_128', 'var_131', 'var_133', 'var_136', 'var_139',
       'var_140', 'var_142', 'var_145', 'var_155', 'var_158', 'var_163',
       'var_175', 'var_191', 'var_197', 'var_200', 'var_212', 'var_216',
       'var_222', 'var_225', 'var_233', 'var_238', 'var_242', 'var_248',
       'var_250', 'var_255', 'var_268', 'var_271', 'var_282', 'var_283',
       'var_289', 'var_297', 'var_298', 'var_301', 'var_305', 'var_310',
       'var_314', 'var_318', 'var_322', 'var_325', 'var_326', 'var_333',
       'var_335', 'var_338', 'var_345', 'var_348', 'var_352', 'var_361',
       'var_362', 'var_366', 'var_367', 'var_369', 'var_378', 'var_381',
       'var_383', 'var_385', 'var_386', 'var_387', 'var_389', 'var_392',
       'var_393', 'var_405', 'var_407', 'var_419', 'var_421', 'var_428',
       'var_429', 'var_435', 'var_446', 'var_448', 'var_452', 'var_456',
       'var_458', 'var_475', 'var_480', 'var_481', 'var_484', 'var_486',
       'var_491', 'var_492', 'var_511', 'var_518', 'var_521', 'var_523',
       'var_525', 'var_527', 'var_540', 'var_546', 'var_549', 'var_551',
       'var_556', 'var_560', 'var_561', 'var_562', 'var_565', 'var_573',
       'var_576', 'var_579', 'var_580', 'var_585', 'var_586', 'var_590',
       'var_591', 'var_593', 'var_602', 'var_610', 'var_615', 'var_624',
       'var_628', 'var_641', 'var_650', 'var_652', 'var_656', 'var_658',
       'var_659', 'var_667', 'var_673', 'var_675', 'var_686', 'var_689',
       'var_691', 'var_692', 'var_695', 'var_698', 'var_714', 'var_715',
       'var_718', 'var_721', 'var_726', 'var_730', 'var_736', 'var_739',
       'var_741', 'var_751', 'var_767', 'var_785', 'var_792', 'var_798',
       'var_799', 'var_801', 'var_802', 'var_807', 'var_820', 'var_823',
       'var_828', 'var_831', 'var_836', 'var_838', 'var_841', 'var_842',
       'var_843', 'var_849', 'var_855', 'var_856', 'var_875', 'var_883',
       'var_886', 'var_890', 'var_893', 'var_896', 'var_897', 'var_899',
       'var_904', 'var_918', 'var_922', 'var_924', 'var_926', 'var_932',
       'var_935', 'var_940', 'var_942', 'var_946', 'var_950', 'var_956',
       'var_974', 'var_976', 'var_980', 'var_985', 'var_987', 'var_993',
       'var_997', 'var_1005', 'var_1017', 'var_1019', 'var_1020'],
      dtype=object)


In [ ]:
features = sorted(features_sel, reverse = False, key = lambda ele: int(ele.split('_')[1]))
x_ext_new = X_ext_infos_df[features]
x_int_new = X_int_infos_df[features]

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
import lightgbm as lgb

params = model_base.get_params()

# Definir o número de rounds de y-randomization
n_rounds = int(50)

# Inicializar dataframe para armazenar os resultados
resultados = pd.DataFrame(columns=['Round', 'Accuracy', 'MCC'])

# Realizar os rounds de y-randomization
for i in range(n_rounds):
    # Embaralhar as variáveis ​​dependentes no dataset de treino e ajustar o modelo
    y_teste_random = np.random.permutation(y_train)

    model_y = lgb.LGBMClassifier(**params)
    model_y.fit(x_train, y_teste_random)

    y_pred = model_y.predict(x_test)

    # Calcular as métricas de desempenho
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    # Apender os resultados ao dataframe
    resultados.loc[i] = [i+1, accuracy, mcc]

# Exibir o dataframe com os resultados
resultados.to_excel(save_path, index=False)
print(resultados)